In [1]:
!pip install -q polars[all]

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.1 which is incompatible.
jupyterlab 4.0.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.1 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.1 requires jupyterlab<5.0.0a0,>=4.0.6, but you have jupyterlab 4.0.5 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.24.3 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.11.4 which is incompatible.
pytoolconfig 1.2.6 requires packaging>=22.0, but you have packaging 21.3 which is incompatible.
spopt 0.6.0 requires shapely

In [2]:
import polars as pl
import json
from tqdm.notebook import tqdm

# Define merge and search conditions
In this case, we are merging using Case ID and searching for Cyclosporin. FAERS also has multiple other keys for identification across tables, like ISR and Primary ID. 

In [3]:
TARGET_ID = 'CASEID'
TARGET_DRUG = 'CYCLOSPORINE'

# Create lazy frames
Next, we create lazy frames to select the adverse effect that ocurred when the target drug was administered. Please note that in this case, the adverse effect also depends on the other drugs that were administered with the target drug. 
For the sake of simplicity, we are not doing that right now. 

In [4]:
drugs_lf = pl.scan_csv(
    source="/kaggle/input/faers-parsed/DRUG.csv",
    separator="\t",
).select(
    [TARGET_ID, 'DRUGNAME']
).filter(pl.col(TARGET_ID).is_not_null() & pl.col('DRUGNAME').str.contains(TARGET_DRUG))
# %%
react_lf = pl.scan_csv(
    source="/kaggle/input/faers-parsed/REACTION.csv",
    separator="\t",
).select(
    [TARGET_ID, 'PT']
).filter(pl.col(TARGET_ID).is_not_null())

# Final result
We simply join the lazy frames and collect the data, and then dump it to the disk.

In [5]:
drug_se_lf = drugs_lf.join(react_lf, on=TARGET_ID).collect()
drug_se_lf = drug_se_lf.select(['DRUGNAME', 'PT']).unique()
print(drug_se_lf)
drug_se_lf.write_csv("dump.tsv", separator="\t")


shape: (11_356, 2)
┌──────────────┬───────────────────────────┐
│ DRUGNAME     ┆ PT                        │
│ ---          ┆ ---                       │
│ str          ┆ str                       │
╞══════════════╪═══════════════════════════╡
│ CYCLOSPORINE ┆ Discomfort                │
│ CYCLOSPORINE ┆ Haemodynamic instability  │
│ CYCLOSPORINE ┆ Perineal abscess          │
│ CYCLOSPORINE ┆ Pyelonephritis            │
│ …            ┆ …                         │
│ CYCLOSPORINE ┆ Dyshidrotic eczema        │
│ CYCLOSPORINE ┆ Hepatic ischaemia         │
│ CYCLOSPORINE ┆ Endocarditis enterococcal │
│ CYCLOSPORINE ┆ Injection site haematoma  │
└──────────────┴───────────────────────────┘


# Looking at target drug partners
Idea is to find the set of drugs that were administered together and the issues they caused.

In [6]:
# Select the case ids that have the target drug

ids_lf = pl.scan_csv(
    source="/kaggle/input/faers-parsed/DRUG.csv",
    separator="\t",
).select(
    [TARGET_ID, 'DRUGNAME']
).filter(pl.col(TARGET_ID).is_not_null() & pl.col('DRUGNAME').str.contains(TARGET_DRUG)).select([TARGET_ID]).unique().collect()


In [7]:
ids = ids_lf.to_series().to_list()
print(len(ids))

26358


In [8]:
# for each id, look up the other drugs and the possible reactions. 
# i = ids[2] # considering a single id before going into a loop. 

adrs = {}

drugs_lf = pl.scan_csv(
        source="/kaggle/input/faers-parsed/DRUG.csv",
        separator="\t",
        ).select(
            [TARGET_ID, 'DRUGNAME']
        )

react_lf = pl.scan_csv(
    source="/kaggle/input/faers-parsed/REACTION.csv",
    separator="\t",
    ).select(
    
        [TARGET_ID, 'PT']
    )

for i in tqdm(ids[:20]):
    co_drugs = drugs_lf.filter(pl.col(TARGET_ID) == i).unique().collect().select(['DRUGNAME']).to_series().to_list()
    co_react = react_lf.filter(pl.col(TARGET_ID) == i).select(['PT']).collect().to_series().to_list()
    adrs[i] = {
        'drugs': co_drugs,
        'reactions': co_react
    }

  0%|          | 0/20 [00:00<?, ?it/s]

In [9]:
with open( 'dump.json', 'w') as f:
   json.dump(adrs, f, indent=4)